In [2]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression,PassiveAggressiveClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score

/home/anirudh/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from keras.utils import np_utils

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [6]:
len(train_df.Event.unique())

6472

In [7]:
train_df.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766787 entries, 0 to 766786
Data columns (total 3 columns):
PID      766787 non-null int64
Date     766787 non-null int64
Event    766787 non-null object
dtypes: int64(2), object(1)
memory usage: 17.6+ MB


In [9]:
#train_df2 = pd.get_dummies(train_df)

In [10]:
train_df.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [11]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(list(train_df['Event'].values)) 
train_df['Event'] = lbl.transform(list(train_df['Event'].values))
print('Shape train: {}\n'.format(train_df.shape))

Shape train: (766787, 3)



In [12]:
dataset=train_df.values
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

(513747, 253040)


In [13]:
#Y_train = np_utils.to_categorical(train, 6472)
#Y_test = np_utils.to_categorical(test, 6472)

In [14]:
train

array([[1028890,  201101,     657],
       [1028890,  201101,    3763],
       [1028890,  201101,    2671],
       ..., 
       [1016736,  201104,    1702],
       [1016736,  201104,    1118],
       [1016736,  201105,    1702]])

In [15]:
test

array([[1016736,  201105,    1118],
       [1016736,  201105,    1054],
       [1016736,  201106,    1568],
       ..., 
       [1022024,  201308,    5078],
       [1022024,  201308,    5095],
       [1022024,  201308,    5072]])

In [16]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), [0,2]]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 2])
    return np.array(dataX), np.array(dataY)

In [17]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [18]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0],  2))
testX = np.reshape(testX, (testX.shape[0], 2))

In [19]:
trainX

array([[1028890,     657],
       [1028890,    3763],
       [1028890,    2671],
       ..., 
       [1016736,    4048],
       [1016736,    2190],
       [1016736,    1702]])

In [20]:
trainY

array([3763, 2671,  678, ..., 2190, 1702, 1118])

In [21]:
testX

array([[1016736,    1118],
       [1016736,    1054],
       [1016736,    1568],
       ..., 
       [1022024,    5089],
       [1022024,    5058],
       [1022024,    5078]])

In [22]:
testY

array([1054, 1568, 1054, ..., 5058, 5078, 5095])

In [23]:
trainX.shape

(513745, 2)

In [24]:
testX.shape

(253038, 2)

In [25]:
trainY.shape

(513745,)

In [26]:
testY.shape

(253038,)

In [27]:
#clf = OneVsRestClassifier(DecisionTreeClassifier())
clf=XGBClassifier()

In [ ]:
score_XG_L1 = cross_val_score(clf,trainX,trainY,cv=2,n_jobs=-1)
print(score_XG_L1)

/home/anirudh/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
Process PoolWorker-9:
Process PoolWorker-10:
Process PoolWorker-5:
Process PoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-8:
Process PoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/anirudh/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-6:
Traceback (most recent call last):
Process PoolWorker-7:
Traceback (most recent call last):
  File "/home/anirudh/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/anirudh/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most re

In [ ]:
y_pred = cross_val_predict(clf,trainX,trainY,cv=7,n_jobs=-1)

from sklearn.metrics import mean_squared_error,r2_score

print(r2_score(trainY,y_pred))
print(mean_squared_error(trainY,y_pred))

In [ ]:
TEST_DATA=train_df.groupby('PID').first().reset_index()

In [ ]:
TEST_DATA.head()